# Custom Evaluation Criteria

In this example, we will see how to create a custom metric.

We will create a metric that evaluates whether a user query is decomposed into sub-queries, covering all the angles of the original query to retrieve all the necessary information.

# Creating the metric

`flow-judge` makes it easy to create custom metrics.

In [1]:
from flow_judge.metrics import CustomMetric, RubricItem
from IPython.display import Markdown, display

# Define the criteria
evaluation_criteria = """Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?"""

# Define the rubric using RubricItem's
rubric = [
    RubricItem(
        score=1,
        description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."),
    RubricItem(
        score=2,
        description="The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic."),
    RubricItem(
        score=3,
        description="The sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. They break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented in the sub-queries, allowing for a thorough and well-rounded investigation of the subject matter."),
]

# Create the metric
sub_query_coverage = CustomMetric(
    name="sub-query-coverage",
    criteria=evaluation_criteria,
    rubric=rubric
)

As you can see, we quickly created a custom metric that will instruct the model to evaluate according to the criteria and rubric we set.

In [2]:
sub_query_coverage

CustomMetric(name='sub-query-coverage', criteria='Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?', rubric=[RubricItem(score=1, description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."), RubricItem(score=2, description='The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic.'), RubricItem(score=3, description='The sub-queries demonstrate exce

# Running evaluations with the custom metric

Now, we just need to initialize the judge with our custom metric and run the evaluations.

In [3]:
from flow_judge.models.model_factory import ModelFactory
from flow_judge.flow_judge import EvalInput, FlowJudge

# Create a model using ModelFactory
model = ModelFactory.create_model("Flow-Judge-v0.1-AWQ")

# Initialize the judge
judge = FlowJudge(metric=sub_query_coverage, model=model)

INFO 09-17 13:42:26 awq_marlin.py:89] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 09-17 13:42:26 config.py:378] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-17 13:42:26 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_back

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-17 13:42:29 model_runner.py:926] Loading model weights took 2.1861 GB
INFO 09-17 13:42:30 gpu_executor.py:122] # GPU blocks: 3085, # CPU blocks: 682


In [4]:
# Sample to evaluate

query = "I placed an order for a custom-built gaming PC (order #AC-789012) two weeks ago, but the estimated delivery date has changed twice since then. Originally it was supposed to arrive yesterday, then it got pushed to next week, and now the tracking page shows 'Status: Processing' with no delivery estimate at all. I've tried calling customer service, but after waiting on hold for an hour, I was told to check the website. Can you please look into this and explain what's causing the delays, when I can realistically expect my order to arrive, and whether I'm eligible for any kind of compensation or expedited shipping given these repeated delays? I'm especially concerned because I need this computer for an upcoming gaming tournament I'm participating in next month."

# bad decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output="""What is the current shipping status of my order? When will my computer arrive?
"""
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.38s/it, est. speed input: 181.28 toks/s, output: 49.41 toks/s]


In [5]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries fail to provide sufficient breadth to cover all aspects of the main query. The main query contains multiple complex issues: repeated delivery date changes, a current 'Status: Processing' on the tracking page, unsuccessful attempts to contact customer service, and a specific concern about needing the PC for an upcoming gaming tournament.

The sub-queries "What is the current shipping status of my order?" and "When will my computer arrive?" only address two very narrow aspects of the main query. They do not touch on the reasons for the delivery delays, the customer service experience, or the possibility of compensation or expedited shipping.

Furthermore, the sub-queries do not address the urgency of the situation due to the upcoming gaming tournament. They also fail to capture the frustration with the repeated delivery date changes and the unsuccessful attempts to contact customer service.

Using these sub-queries alone would result in a severely limited exploration of the topic. They would not allow for a comprehensive investigation of the multiple issues raised in the main query, leaving many important aspects unexplored.

__Score:__
1

In [6]:
# Good decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output="""1. What is the current status of order #AC-789012, and why has it changed from the original estimated delivery date?
2. What specific factors are causing the delays in processing and shipping this custom-built gaming PC?
3. Based on the current situation, when can the customer realistically expect the order to arrive?
4. Given the repeated delays and changes in estimated delivery, what compensation options (if any) are available to the customer?
5. Is expedited shipping an option at this point, and if so, how would it affect the delivery timeline?
6. How can the urgency of this order be communicated and prioritized, considering the customer's upcoming gaming tournament next month?
7. What steps has customer service already taken to address this issue, and what additional actions can be taken to resolve it?
8. How can the customer receive more frequent and accurate updates about their order status going forward?"""
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.52s/it, est. speed input: 155.43 toks/s, output: 47.91 toks/s]


In [7]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. Each significant facet of the main query is represented in the sub-queries, ensuring a comprehensive exploration of the topic.

1. The first sub-query addresses the current status and reasons for the change in the estimated delivery date, covering the aspect of delays and their causes.
2. The second sub-query delves into the specific factors causing the delays, providing a deeper understanding of the situation.
3. The third sub-query focuses on the realistic expectation of the order's arrival, addressing the customer's concern about the delivery timeline.
4. The fourth sub-query explores compensation options, directly addressing the customer's question about eligibility for compensation.
5. The fifth sub-query considers expedited shipping as an option, which is relevant given the repeated delays.
6. The sixth sub-query addresses the urgency of the order in relation to the customer's upcoming gaming tournament, ensuring this critical aspect is not overlooked.
7. The seventh sub-query examines the actions taken by customer service and potential additional steps, providing insight into the company's response to the issue.
8. The eighth sub-query suggests ways to improve communication about order status, which is relevant for future interactions.

Overall, these sub-queries provide a thorough and well-rounded investigation of the subject matter, covering all major aspects of the main query without any noticeable gaps.

__Score:__
3

In [8]:
# Okish decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output=""""1. What is the current status of order #AC-7892?
Why has the delivery date changed multiple times?
When will the custom-built gaming PC actually arrive?
What are your store hours on weekends?
How can I get better customer service support?
8. Can the order be expedited?
9. What is the name of the person that is responsible for the company?
"""
)

# Run the evaluation
result = judge.evaluate(eval_input)
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.84s/it, est. speed input: 152.98 toks/s, output: 48.36 toks/s]


__Feedback:__
The generated sub-queries partially address the main query but lack comprehensive breadth. They touch on several important aspects of the main query, such as the current status of the order, reasons for delivery date changes, expected arrival time, and the possibility of expediting the order. However, there are noticeable gaps in coverage.

The sub-queries fail to address some critical aspects of the main query:
1. They don't specifically address the repeated delays and their impact on the customer's situation.
2. They don't mention the customer's need for the PC for an upcoming gaming tournament.
3. They don't address the customer's eligibility for compensation due to repeated delays.
4. They don't address the customer's frustration with the customer service experience.

While the sub-queries cover some dimensions of the main query, they miss several important facets. For example, the sub-query about store hours on weekends is irrelevant to the main query's concerns. The sub-query about the person responsible for the company doesn't address the customer's issues with the order or delivery.

Answering these sub-queries would provide a partial exploration of the main topic, but significant aspects of the customer's concerns would remain unaddressed. Therefore, the sub-queries lack the comprehensive breadth required to fully explore all aspects of the main query.

__Score:__
2